# Funciones para encontrar usuarios a testear

In [1]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
from tqdm import tqdm
import pickle

In [2]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

In [3]:
# targetUsers = pd.DataFrame(columns=['username'])

targetUsers = pd.read_pickle('../datasets/targetUsersDataset2.pkl').head(1)
print(len(targetUsers))
targetUsers.head()

1


,username
235491971,gsrcaballero


In [4]:
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getNewPage(term, searchType, maxPosition):
    if(searchType != 'q'):
        term = '#'+term
    pageResponse = requests.get(
        'https://twitter.com/i/search/timeline',
        params={
            'vertical': 'news',
            'q': term,
            'max_position': maxPosition,
            'include_entities': 1,
            'include_available_features': 1,
            'reset_error_state': False
        },
        proxies=proxies,
        headers=headersMore
    )
    return(pageResponse.json())
#     print(pageResponse.json().keys())
#     print(pageResponse.json()['min_position'])
#     display(BeautifulSoup(pageResponse.json()['items_html'], 'html.parser'))
    return pageResponse.json()['min_position'], BeautifulSoup(pageResponse.json()['items_html'], 'html.parser')

def getUserTweetData(tweet):
    try:
        username = tweet.findAll(attrs={'class':'username'})[0].text[1:]
        userId = tweet.findAll('div', attrs={'class':'js-stream-tweet'})[0].attrs['data-user-id']
        targetUsers.loc[userId] = [username]
    except:
        print('ERROR---------------------')
        print(tweet)
    return [username, userId]

def searchTerm(term, searchType='q', maxPages=15):
    pageResponse = False
    if(searchType == 'q'):
        pageResponse = requests.get('https://twitter.com/search?q='+term, headers=headersChrome, proxies=proxies)
    else:
        pageResponse = requests.get('https://twitter.com/hashtag/'+term, headers=headersChrome, proxies=proxies)
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)
    for tweet in tweetList:
        getUserTweetData(tweet)
    nextKey = dataMax
    for pageNum in tqdm(range(maxPages)):
        response = getNewPage(term, searchType, nextKey)
        nextKey = response['min_position']
        htmlResponse = BeautifulSoup(response['items_html'], 'html.parser')
        if(len(htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)) == 0):
            print(response)
            print('NO MORE',htmlResponse)
            break
        for tweet in htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False):
            getUserTweetData(tweet)

# searchTerm('cfk', '#', 15)
# searchTerm('cambiemos', '#', 5)
# searchTerm('MacriVendepatria', '#', 5)
# searchTerm('JuntosPorElCambio', '#', 5)
# searchTerm('carrio', 'q', 100)

In [5]:
import threading
lock = threading.RLock()
def process_id(id):
    """process a single ID"""
    # fetch the data
    r = requests.get(url_t % id)
    # parse the JSON reply
    data = r.json()
    # and update some data with PUT
    requests.put(url_t % id, data=data)
    return data

def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in id_range:
        store[id] = searchTerm(id, 'q', 600)
    return store

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        with lock:
            t = Thread(target=process_range, args=(ids,store))
            threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    print('Done')
    print(len(targetUsers))
    display(targetUsers.head())
    return 'Done'

termsToSearch = [
    '%23HayEsperanza',
    '%23FernandezFernandez2019',
    '%23ElCircoDeMariuYMau',
    '%23AlbertoPresidente',
    'TOD ☀️S',
    '%23EsConTodos',
    '%23HayOtroCamino',
    '%23MacriTeQuedaPoco',
    '%23AltaSuciedadPRO'
    
]

threaded_process_range(8, termsToSearch)
# filename = 'targetUsersDataset.pkl'
# with open(filename, 'wb') as twUsersTarget:
#     pickle.dump(targetUsers, twUsersTarget)
    
# termsToSearch = [
#     'macri FMI',
#     'sinceramente falso',
#     'ella miente',
#     'massa',
#     'cristina pobreza'
# ]
# threaded_process_range(8, termsToSearch)
# filename = 'targetUsersDataset.pkl'
# with open(filename, 'wb') as twUsersTarget:
#     pickle.dump(targetUsers, twUsersTarget)

  0%|          | 0/600 [00:00<?, ?it/s]

{'min_position': 'thGAVUV0VFVBaCwLTp4Lrl9B8WgsC06eC65fQfEhhsEmPC6wAAAfQ_gGJN0vGp_AAAAAsP9Cf2-pRgAA_0yusGlpABD_QkSHxUIAYP9CLkSVQQAQ_0PwSOFPABD-w2uYSWsAIP89A9RlTAAA_0Jc7c1HAAD_TKegTWkAAP9CatiNVQAA_0KXALFCABFQAVABUAJQBVABUAAA==', 'has_more_items': False, 'items_html': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n', 'new_latent_count': 0, 'focused_refresh_interval': 30000}
NO MORE 



ERROR---------------------
<li class="js-stream-item stream-item stream-item" data-item-id="1147078668470104065" data-item-type="tweet" id="stream-item-tweet-1147078668470104065">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content" data-component-context="tweet" data-conversation-id="1147078668470104065" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1147078668470104065" data-name="Silviar6566" data-permalink-path="/silviar6566/status/1147078668470104065" data-reply-to-users-json='[{"id_str":"1102751321785098240","screen_name":"silviar6566","name":"Silviar6566","emojified_name":{"text":"Silviar6566","emojified_text_as_html":"Silviar6566"}}]' data-screen-name="silviar6566" data-tweet-id="1147078668470104065" data-tweet-nonce="1147078668470104065-b6cc1ddf-5c36-407d-ae36-570211b4bbcf" data-tweet-stat-initialized="true" data-user-id="11027513217850

  0%|          | 0/600 [00:00<?, ?it/s]

ERROR---------------------
<li class="js-stream-item stream-item stream-item" data-item-id="1149859509730459648" data-item-type="tweet" id="stream-item-tweet-1149859509730459648">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content" data-component-context="tweet" data-conversation-id="1149859509730459648" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1149859509730459648" data-name="AGRUPACIÓN PAMPA" data-permalink-path="/pampaagrupacion/status/1149859509730459648" data-reply-to-users-json='[{"id_str":"798958051114029056","screen_name":"pampaagrupacion","name":"AGRUPACI\u00d3N PAMPA","emojified_name":{"text":"AGRUPACI\u00d3N PAMPA","emojified_text_as_html":"AGRUPACI\u00d3N PAMPA"}}]' data-screen-name="pampaagrupacion" data-tweet-id="1149859509730459648" data-tweet-nonce="1149859509730459648-45c74bb6-503b-43b9-be76-bec5eac5e5cb" data-tweet-stat-i

...

{'min_position': 'thGAVUV0VFVBaAwLXx0oX06B8WgoC7ye_U_vQfEhjIBQQ9VDthRbvUAAAAAAAAFXwAAAAHAAAAVgRIIIiggCyFmAQAAQIBgRAkgBIIIQBcIyAAmAICwEwACIIGBAAJCR4oAByIJBBADAxSAREQCJREQCAwARQEoYSAABAAjAAgwQIhkAACABUEAQQQAUggAAiIJAFQKAQAAAQVQABADagAAQUAABBCACgJsERAiQgkIEGTABABIeRAAgIYAQDAAgAUgIAQIAsCAAACBxAdBiiQHQgQwAUDACARBAAaiEDQAAJBAQJYAEAEAhAIGCiAtCJCgiAAJgIAAC4QIQJAQBghAACkFAwAAIAKjgAAEMcJCEREAEKicMTklQtGQAohCIIChgQGIAAABABUKABEUAQCAhUABAlYCAgABEQIAAAAAQgJBAgAJDAAUgQEUYAgAqBAQAgBiFIgUBBAawDYYFgIgCJAgAAGcxAACAwgABAZAgMQNGAAUCEEIBAADAAKCQAAAGQgmAAAAOJygBIgAVwAAgkIEBiAAAEGABAWBADBIACFADmJiABFAYBACIAQADgAAAURgCADgwVAYEAkGZKGAAgAAAEBAQcGEIlABVIAiIhQgASAiiABQIABAgScBAwkwKAAAFQKyAQAkAAgIgDNgORQECAVVAKKAwBKAQ6pQAAhAUCAIEIAABBAICAMjJAQAicAQAYACIBA0wkQRGAAACAABARAAADEIICh2qQBARxiSQAIRSBITWAEJIIIAAgARQAgmCIhgABAQEgHAwDoAaAJCKQAgAVkAEEAIGFCAAAGRiBwBAKMCACQgBQEAECJESGwAoAE5YFAAAgICAFQAYQYgiKoCAEIQBIACIEABggAAEAZIAYBFAEMgJQyAEMAIEVCECQGRAKBEAACCIUgERxAFJQgEAABAAkiCQWAAAQIFAVQGJCBADBTBAAIQEgCgLExFAQkABAggQAHZAIR








  0%|          | 0/600 [00:00<?, ?it/s]






  0%|          | 1/600 [00:01<15:51,  1.59s/it]






  0%|          | 2/600 [00:03<15:18,  1.54s/it]






  0%|          | 3/600 [00:04<14:46,  1.48s/it]






  1%|          | 4/600 [00:05<13:37,  1.37s/it]






  1%|          | 5/600 [00:07<14:23,  1.45s/it]






  1%|          | 6/600 [00:08<13:58,  1.41s/it]






  1%|          | 7/600 [00:09<13:07,  1.33s/it]






  1%|▏         | 8/600 [00:10<13:07,  1.33s/it]






  2%|▏         | 9/600 [00:11<11:57,  1.21s/it]






  2%|▏         | 10/600 [00:12<10:51,  1.10s/it]

{'min_position': 'thGAVUV0VFVBaAgKO9ha7k8x8WgIC7wdrAzfQfEhjIBQQ9VDthRbvUAAAAAAAAFXwAAAAHAAAAVgABYACAABCCQQAAABEAAoAABgAAABQEyACEiIQEAQAAAAACAAAEVAIAAQAAAQgAAgCKgBEADQAogZgAgEIIgJIAAIhAAAZABAgABAQghAgBCAAAAAAAgIIAAAAABACACAAIQAAAAQQgCAECAABQAAQEAgjAAAAQABAJAEAEAAYFCAAACAQQABAABBAYAgAAAIgQBAAIEAQAABAgEAIGggAgwgkgggmgAgAMRAACAIwAABoYAQAAgEAAACIQQgAABAQAYIACEBEEIQSAIYAFAABBAQgEAIAAwjBQASRUAkACAACACgEAAgABAEAgBAECAAgC0EAAhCIAAHmSAAAAFAQAAABAFQChAACgCAFAAIIARAhAIAGAGAQQAQQEBAgAQpABiEQEAAAAAIMAQAAgAADCAAgAQwCAIARSRgALCIEoCgAAAAFAIAACWAAAgAQAAADQAIAEAEFEGEACAswAkwQBAAACAVKAQAQMGRAAEgAAACAEAPCCAAAAAIAAJAL0gAAEAQAAQgAABAAAAAAFACAAGAAIBQACAAMACRBAFEEIAsQBAYEAAgAQAAAAAAAABgQoBCAAMIAELABQgJEABASAAAAQBAMAQAAYFQwCASAwRoKAqAIAAACwEAAAEABEggBAMBABCCAgAgQQICgBAIgEBAiAgASBDBCAAgBgAGhgAAoABEAiFQBBABAAsVAkEAQAoBBAaAQAAAAGAAQKABAAhgAAEhIAAADgSAkAiggAAAJABCAgQgiAAAIAAIAIEAAAAAAEBAgECAIAAAICAAABBBBUBABAGCAAAAUK0QEQAABCQIAAoQCIIECAHAAJCEQCACAIIAhAAAQAAAAAAIYABCMAEQgAQAQoIAEAQAJEIBAAFcCEGAEAADAGAACAEBAQAgBRIBAC

,username
235491971,gsrcaballero
325018071,hector06gramajo
369120654,wadodecorrido
1127110904024305664,eltanomdp86
1034895558,Marelvigo


'Done'

In [6]:
len(targetUsers)

771

In [7]:
targetUsers

,username
235491971,gsrcaballero
325018071,hector06gramajo
369120654,wadodecorrido
1127110904024305664,eltanomdp86
1034895558,Marelvigo
931986881880973313,Sergiosueldoo
1266161713,Ciro_alejandro1
958345618669428736,Victori56985566
244356927,vtolosapaz
169290793,valentinanst


In [8]:
#guardar resultados
import pickle
filename = '../datasets/targetUsersDataset_k.pkl'
with open(filename, 'wb') as twUsersTarget:
    pickle.dump(targetUsers, twUsersTarget)